In [10]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from ipywidgets import interact

# Size of the 3D grid
N = 30

# Time step
dt = 0.1

# Initialize U and V
U = np.random.normal(0.50, 0.10, (N, N, N))  # Random initial condition for U
V = np.random.normal(0.20, 0.10, (N, N, N))  # Random initial condition for V

# Parameters
Du, Dv, F, k = 0.16, 0.08, 0.055, 0.062 # Adjusted parameters for more dynamic pattern

# Function to calculate Laplacian using convolution
def laplacian(Z):
    laplacian_kernel = np.array([
        [[0.05, 0.2, 0.05],
         [0.2, -1, 0.2],
         [0.05, 0.2, 0.05]],
        
        [[0.2, 0.8, 0.2],
         [0.8, -4, 0.8],
         [0.2, 0.8, 0.2]],
        
        [[0.05, 0.2, 0.05],
         [0.2, -1, 0.2],
         [0.05, 0.2, 0.05]]
    ])
    return signal.convolve(Z, laplacian_kernel, mode='same', method='direct')

# Store each frame of the simulation
frames = []

# Run the simulation
for i in range(100):  # 100 steps
    U_new = U + dt * (Du * laplacian(U) - U * V * V + F * (1 - U))
    V_new = V + dt * (Dv * laplacian(V) + U * V * V - (F + k) * V)
    U, V = np.clip(U_new, 0, 1), np.clip(V_new, 0, 1)
    if i % 10 == 0:  # Store every 10th frame
        frames.append(V.copy())  # Store the current frame

fig = plt.figure()
plt.close()

def display_frame(i):
    fig.clear()
    ax = fig.add_subplot(111, projection='3d')
    print('Max: %f' % np.max(frames[i]))
    print('Min: %f' % np.min(frames[i]))
    print('Mean: %f' % np.mean(frames[i]))
    ax.voxels(frames[i] > 0.3, facecolors='b', alpha=1)
    ax.set_title('Frame: %d' % i)
    display(fig)

# Use a slider to navigate through the frames
interactive_plot = interact(display_frame, i=(0, len(frames)-1))


interactive(children=(IntSlider(value=4, description='i', max=9), Output()), _dom_classes=('widget-interact',)…